In [1]:
!pip install backtrader
!pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 7.3 MB/s eta 0:00:00


In [2]:
import backtrader as bt
import math
import yfinance as yf

RSI 지표 과매수(70)시 매도 청산,  RSI 지표 과매도(30)시 매수 진입

In [3]:
class RSI(bt.Strategy):
    params = dict(
        rsi_period=14,  # RSI 기간 설정
        rsi_low=30,  # RSI low 설정
        rsi_high=70  # RSI high 설정
    )
    def __init__(self):
        # RSI 지표 정의
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
        # 매수 신호 추적 변수
        self.order = None

    def next(self):
        if self.order:  # 만약 주문이 있으면 대기
            return
        # 포지션이 없을 때(시장 미진입 상태)
        if not self.position:
            # RSI < 30이면 매수 진입 시도
            if self.rsi[-1] <= self.params.rsi_low < self.rsi[0]:  # RSI가 rsi_low보다 낮은지 확인하고 매수
                order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
                self.buy(size=order_size)

        # 포지션이 있을 때(시장 진입 상태)
        else:
            if self.rsi[-1] >= self.p.rsi_high > self.rsi[0]:  # RSI가 rsi_high보다 높은지 확인하고 매도
                self.close()

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')


In [4]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()
    cerebro.addstrategy(RSI)
    cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
    cerebro.broker.setcash(10_000_000)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
    # 열 이름 전처리
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


Starting Portfolio Value: 10000000.00
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


2020-03-03 [매수 주문 실행] 종목:  	 수량: 1116 	 가격: 8865.39
2020-03-28 [매도 주문 실행] 종목:  	 수량: -1116 	 가격: 6467.25
2020-09-05 [매수 주문 실행] 종목:  	 수량: 684 	 가격: 10512.53
2020-09-21 [매도 주문 실행] 종목:  	 수량: -684 	 가격: 10934.93
2021-04-27 [매수 주문 실행] 종목:  	 수량: 137 	 가격: 54030.30
2021-08-11 [매도 주문 실행] 종목:  	 수량: -137 	 가격: 45599.70
2021-09-22 [매수 주문 실행] 종목:  	 수량: 153 	 가격: 40677.95
2021-10-23 [매도 주문 실행] 종목:  	 수량: -153 	 가격: 60694.63
None 주문이 거부되었습니다. 종목:  	 수량: 161 	 가격: 0.00
2021-11-28 [매수 주문 실행] 종목:  	 수량: 168 	 가격: 54813.02
2022-02-12 [매도 주문 실행] 종목:  	 수량: -168 	 가격: 42412.30
2022-02-25 [매수 주문 실행] 종목:  	 수량: 186 	 가격: 38333.75
2022-04-03 [매도 주문 실행] 종목:  	 수량: -186 	 가격: 45859.13
2022-04-20 [매수 주문 실행] 종목:  	 수량: 204 	 가격: 41501.75
2022-08-10 [매도 주문 실행] 종목:  	 수량: -204 	 가격: 23162.90
2022-09-03 [매수 주문 실행] 종목:  	 수량: 236 	 가격: 19969.72
2022-10-28 [매도 주문 실행] 종목:  	 수량: -236 	 가격: 20287.96
2022-11-11 [매수 주문 실행] 종목:  	 수량: 271 	 가격: 17583.25
2022-12-15 [매도 주문 실행] 종목:  	 수량: -271 	 가격: 17813.64
2022-12-30 

스탑로스 10%

In [5]:
class RSIStopLoss(bt.Strategy):
    params = dict(
        rsi_period=14,  # RSI 기간 설정
        rsi_low=30,  # RSI low 설정
        rsi_high=70,  # RSI high 설정
        stop_loss=0.1 # 손절 비율(10%)
    )
    def __init__(self):
        # RSI 지표 정의
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
        # 매수 신호 추적 변수
        self.order = None
        # 매수 가격 저장
        self.buy_price = None

    def next(self):
        if self.order:  # 만약 주문이 있으면 대기
            return
        # 포지션이 없을 때(시장 미진입 상태)
        if not self.position:
            # RSI < 30이면 매수 진입 시도
            if self.rsi[-1] <= self.params.rsi_low < self.rsi[0]:  # RSI가 rsi_low보다 낮은지 확인하고 매수
                order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
                self.buy(size=order_size)
                self.buy_price = self.data.close[0]

        # 포지션이 있을 때(시장 진입 상태)
        else:
            # RSI가 rsi_high보다 높거나 매수가 대비 10% 하락하면 매도
            stop_loss_price = self.buy_price * (1 - self.params.stop_loss) # 손절가 계산
            if self.rsi[-1] >= self.p.rsi_high > self.rsi[0] or self.data.close[0] < stop_loss_price:
                self.close()
                self.buy_price = None # 매도 후 매수 가격 초기화

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')


In [6]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()
    cerebro.addstrategy(RSIStopLoss)
    cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
    cerebro.broker.setcash(10_000_000)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
    # 열 이름 전처리
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


Starting Portfolio Value: 10000000.00

[*********************100%***********************]  1 of 1 completed

2020-03-03 [매수 주문 실행] 종목:  	 수량: 1116 	 가격: 8865.39
2020-03-10 [매도 주문 실행] 종목:  	 수량: -1116 	 가격: 7922.15
2020-03-23 [매수 주문 실행] 종목:  	 수량: 1509 	 가격: 5831.37
2020-03-28 [매도 주문 실행] 종목:  	 수량: -1509 	 가격: 6467.25
2020-09-05 [매수 주문 실행] 종목:  	 수량: 922 	 가격: 10512.53
2020-09-21 [매도 주문 실행] 종목:  	 수량: -922 	 가격: 10934.93
2021-04-27 [매수 주문 실행] 종목:  	 수량: 185 	 가격: 54030.30
2021-05-16 [매도 주문 실행] 종목:  	 수량: -185 	 가격: 46716.64
2021-05-19 [매수 주문 실행] 종목:  	 수량: 201 	 가격: 42944.98
2021-05-20 [매도 주문 실행] 종목:  	 수량: -201 	 가격: 36753.67
2021-05-27 [매수 주문 실행] 종목:  	 수량: 187 	 가격: 39316.89
2021-05-30 [매도 주문 실행] 종목:  	 수량: -187 	 가격: 34607.41
2021-05-31 [매수 주문 실행] 종목:  	 수량: 180 	 가격: 35658.59
2021-06-22 [매도 주문 실행] 종목:  	 수량: -180 	 가격: 31622.38
2021-07-22 [매수 주문 실행] 종목:  	 수량: 176 	 가격: 32138.87
2021-08-11 [매도 주문 실행] 종목:  	 수량: -176 	 가격: 45599.70
2021-09-22 [매수 주문 실행] 종목:  	 수량: 196 	 가격: 40677.95
2021-10-23 [매도 주문 실행] 종목:  	 수량: -196 	 가격: 60694.63
None 주문이 거부되었습니다. 종목:  	 수량: 206 	 가격: 0.00
2021-11-28 

스탑로스10% + 윌리엄스 리스크고정

In [7]:
import math
import backtrader as bt

class RSIStopLossRiskFixed(bt.Strategy):
    params = dict(
        rsi_period=14,     # RSI 기간 설정
        rsi_low=30,        # RSI 진입(low) 임계값
        rsi_high=70,       # RSI 청산(high) 임계값
        stop_loss=0.1,     # 손절 비율 (10%)
        risk_percent=0.03, # 계좌 가치 대비 리스크 비율 (3%)
    )

    def __init__(self):
        # RSI 지표 정의
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
        # 주문 상태 추적 변수
        self.order = None
        # 매수 가격 저장
        self.buy_price = None

    def next(self):
        if self.order:  # 미체결 주문이 있으면 대기
            return

        # 포지션이 없을 때 (시장 미진입 상태)
        if not self.position:
            # RSI가 이전에 rsi_low 이하였다가 현재 rsi가 rsi_low 이상으로 상승하는 경우 진입
            if self.rsi[-1] <= self.params.rsi_low < self.rsi[0]:
                self.buy_price = self.data.close[0]
                # 리스크 금액: 계좌 가치의 risk_percent
                risk_amount = self.broker.get_value() * self.params.risk_percent
                # 손절가: 매수가의 stop_loss 비율 만큼 하락한 가격
                stop_loss_price = self.buy_price * (1 - self.params.stop_loss)
                # 포지션 사이즈 계산: 리스크 금액 / (매수가 - 손절가)
                position_size = risk_amount / (self.buy_price - stop_loss_price)
                position_size = int(position_size)

                if position_size <= 0:
                    return

                self.buy(size=position_size)
        # 포지션이 있을 때 (시장 진입 상태)
        else:
            # 손절가 재계산
            stop_loss_price = self.buy_price * (1 - self.params.stop_loss)
            # RSI가 rsi_high 이상으로 하락 반전하거나, 현재 가격이 손절가 아래로 내려가면 청산
            if self.rsi[-1] >= self.params.rsi_high > self.rsi[0] or self.data.close[0] < stop_loss_price:
                self.close()
                self.buy_price = None  # 청산 후 매수 가격 초기화

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
                )
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
                )
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
            )


In [8]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()
    cerebro.addstrategy(RSIStopLossRiskFixed)
    cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
    cerebro.broker.setcash(10_000_000)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
    # 열 이름 전처리
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2020-03-03 [매수 주문 실행] 종목:  	 수량: 338 	 가격: 8865.39
2020-03-10 [매도 주문 실행] 종목:  	 수량: -338 	 가격: 7922.15
2020-03-23 [매수 주문 실행] 종목:  	 수량: 497 	 가격: 5831.37
2020-03-28 [매도 주문 실행] 종목:  	 수량: -497 	 가격: 6467.25
2020-09-05 [매수 주문 실행] 종목:  	 수량: 284 	 가격: 10512.53
2020-09-21 [매도 주문 실행] 종목:  	 수량: -284 	 가격: 10934.93
2021-04-27 [매수 주문 실행] 종목:  	 수량: 55 	 가격: 54030.30
2021-05-16 [매도 주문 실행] 종목:  	 수량: -55 	 가격: 46716.64
2021-05-19 [매수 주문 실행] 종목:  	 수량: 67 	 가격: 42944.98
2021-05-20 [매도 주문 실행] 종목:  	 수량: -67 	 가격: 36753.67
2021-05-27 [매수 주문 실행] 종목:  	 수량: 70 	 가격: 39316.89
2021-05-30 [매도 주문 실행] 종목:  	 수량: -70 	 가격: 34607.41
2021-05-31 [매수 주문 실행] 종목:  	 수량: 74 	 가격: 35658.59
2021-06-22 [매도 주문 실행] 종목:  	 수량: -74 	 가격: 31622.38
2021-07-22 [매수 주문 실행] 종목:  	 수량: 79 	 가격: 32138.87
2021-08-11 [매도 주문 실행] 종목:  	 수량: -79 	 가격: 45599.70
2021-09-22 [매수 주문 실행] 종목:  	 수량: 70 	 가격: 40677.95
2021-10-23 [매도 주문 실행] 종목:  	 수량: -70 	 가격: 60694.63
2021-11-26 [매수 주문 실행] 종목:  	 수량: 57 	 가격: 58960.29
2021-12-05 [매도 주문 실행